In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os
import re
import numpy as np
from skimage.filters import gaussian
import tifffile
from skimage import io, filters, exposure, morphology, util
from skimage.morphology import binary_closing, binary_opening, ball, remove_small_holes, remove_small_objects
from skimage.measure import label
from scipy import ndimage as ndi
import matplotlib
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure



In [ ]:
# creates a list of paths (path_list) with working files

import os

# Create a list of paths the name of which has two variables: mNRA_Name and SubSET
# def get_path(mNRA_Red, mRNA_Green, SubSET):

def get_path():

    input_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3'
    path_list = []

    # Loop through all directories and subdirectories inside the input directory
    for root, dirs, files in os.walk(input_dir):
        # Loop through all files in the current directory
        for file in files:
            # Check if the file has an _info.txt extension and starts with the folder name
            if "_info.txt" in file and file.startswith(os.path.basename(root)):
                
                # Read the file content and convert it to lowercase
                
                path_list.append(root)

    # Return the list of paths
    return path_list


In [ ]:
# runs function get_path and saves path list as .txt file

path_list = get_path()

with open('path_list.txt', 'w') as file:
    for item in path_list:
        file.write(item + '\n')
        
len(path_list)

In [ ]:
path_list # to check the nimber of folders

In [ ]:
# function for parallelization that takes masked and raw images of mRNA from the path list and labels puncta

def Nikon_segment_mRNA(path):
    
    mRNA_dir = fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3\mRNA'
    
    # Check if the directory exists, and create it if it does not
    if not os.path.exists(mRNA_dir):
        os.makedirs(mRNA_dir, exist_ok=True)
        
    try:
        
        with open(os.path.join(path, os.path.basename(path) + '_info.txt'), 'r', encoding='utf-8', errors='ignore') as file:
            content = file.read()

        # Extract channel numbers and names using regular expressions
        # if you have 3 channels:

        # channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+)', content)[0]))
        # channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+)', content)[0]
        
        # uncomment if you have 4 channels:
        channel_numbers = list(map(int, re.findall(r'ChannelNumbers:(\d+),(\d+),(\d+),(\d+)', content)[0])) #ONLY for 3rd channel
        channel_names = re.findall(r'ChannelNames:(\w+),(\w+),(\w+),(\w+)', content)[0] #ONLY for 3rd channel
        
        
        # Combine channel numbers and names into a dictionary
        channel_info = dict(zip(channel_numbers, channel_names))

        mask0 = channel_names[0]
        raw0 = channel_numbers[0]
        mask1 = channel_names[1]
        raw1 = channel_numbers[1]
        mask2 = channel_names[2]
        raw2 = channel_numbers[2]
        
        mask3 = channel_names[3] #ONLY for 4rd channel
        raw3 = channel_numbers[3] #ONLY for 4rd channel
        


        mRNA1_mask_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw1}.tif')
        mRNA2_mask_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw2}.tif')
        mRNA3_mask_path = os.path.join(path, os.path.basename(path) + f'_ch0{raw3}.tif')

        Body_mask_path = os.path.join(path, 'Segmented\Masked_Body_image.tif')
    
        # Load the raw 3D tif image
        raw_image1 = tifffile.imread(mRNA1_mask_path)
        raw_image2 = tifffile.imread(mRNA2_mask_path)
        raw_image3 = tifffile.imread(mRNA3_mask_path)
        binary_body_image = tifffile.imread(Body_mask_path)
        
        # removes small very objects
        min_size = 60 # parameter to change (20-80)

        # Creating mask
        if mask1 == "Scn5a":
            denoised_mrna1 = ndi.median_filter(raw_image1, size=1)
            smoothed_image1 = gaussian(denoised_mrna1, sigma=1.5) # a parameter to change (1-2)
            threshold_value1 = np.max(smoothed_image1) / 2 # the main parameter to change
            binary_mask1 = smoothed_image1 > threshold_value1 
        else:    
            denoised_mrna1 = ndi.median_filter(raw_image1, size=1)
            smoothed_image1 = gaussian(denoised_mrna1, sigma=1.5) # a parameter to change (1-2)
            threshold_value1 = np.max(smoothed_image1) / 2 # the main parameter to change
            binary_mask1 = smoothed_image1 > threshold_value1 
            
        if mask2 == "ProtRyr2":
            denoised_mrna2 = ndi.median_filter(raw_image2, size=1)
            smoothed_image2 = gaussian(denoised_mrna2, sigma=1.5) # a parameter to change (1-2)
            threshold_value2 = np.max(smoothed_image2) / 4 # the main parameter to change (3-11)
            binary_mask2 = smoothed_image2 > threshold_value2 
            
        if mask2 == "RnaRyr2":
            denoised_mrna2 = ndi.median_filter(raw_image2, size=1)
            smoothed_image2 = gaussian(denoised_mrna2, sigma=2) # a parameter to change (1-2)
            threshold_value2 = np.max(smoothed_image2) / 3.5 # the main parameter to change (3-11)
            binary_mask2 = smoothed_image2 > threshold_value2 

        denoised_mrna3 = ndi.median_filter(raw_image3, size=1)
        smoothed_image3 = gaussian(denoised_mrna3, sigma=1.3) # a parameter to change (1-2)
        threshold_value3 = np.max(smoothed_image3) / 5 # the main parameter to change (3-11)
        binary_mask3 = smoothed_image3 > threshold_value3          

        # Create a binary mask for segmented mRNA
        mrna1 = binary_mask1 * binary_body_image
        mrna2 = binary_mask2 * binary_body_image
        mrna3 = binary_mask3 * binary_body_image
        
        filtered_mrna1 = remove_small_objects(util.img_as_bool(mrna1), min_size=min_size)
        filtered_mrna2 = remove_small_objects(util.img_as_bool(mrna2), min_size=min_size)
        filtered_mrna3 = remove_small_objects(util.img_as_bool(mrna3), min_size=min_size)

        directory_name = 'Segmented'
        segmented_dir_path = os.path.join(path, directory_name)

        # Add these lines
        if not os.path.exists(segmented_dir_path):
            os.makedirs(segmented_dir_path)
        
        Masked_filtered_image1_path = os.path.join(segmented_dir_path, f'Masked_{mask1}_filtered_image.tif')
        Masked_filtered_image2_path = os.path.join(segmented_dir_path, f'Masked_{mask2}_filtered_image.tif')
        Masked_filtered_image3_path = os.path.join(segmented_dir_path, f'Masked_{mask3}_filtered_image.tif')
        io.imsave(Masked_filtered_image1_path, filtered_mrna1)
        io.imsave(Masked_filtered_image2_path, filtered_mrna2)
        io.imsave(Masked_filtered_image3_path, filtered_mrna3)
        
        print(f"Masked image 4 saved in: {os.path.abspath(Masked_filtered_image3_path)}")
        
        # Set up the plot configurations
        
        # One channel. Change parameters accordingly
        # plot_titles = [ f'Raw {mask3} {raw_image3.shape[1]}', f'Binary {mask3} {raw_image3.shape[1]}', f'filtered {mask3} {raw_image3.shape[1]}']
        # images = [np.max(raw_image3, axis=0), np.max(mrna3, axis=0), np.max(filtered_mrna3, axis=0)]
        
        # Two channels. Change parameters accordingly
        # plot_titles = [ f'Raw {mask1} {raw_image1.shape[1]}', f'Binary {mask1} {raw_image1.shape[1]}', f'filtered {mask1} {raw_image1.shape[1]}', f'Raw {mask2}', f'Binary {mask2}', f'filtered {mask2}']
        # images = [np.max(raw_image1, axis=0), np.max(mrna1, axis=0), np.max(filtered_mrna1, axis=0), np.max(raw_image2, axis=0), np.max(mrna2, axis=0), np.max(filtered_mrna2, axis=0)]

        # Three channels
        plot_titles = [ f'Raw {mask1} {raw_image1.shape[1]}', f'Binary {mask1} {raw_image1.shape[1]}', f'filtered {mask1} {raw_image1.shape[1]}', f'Raw {mask2}', f'Binary {mask2}', f'filtered {mask2}', f'Raw {mask3}', f'Binary {mask3}', f'filtered {mask3}']
        images = [np.max(raw_image1, axis=0), np.max(mrna1, axis=0), np.max(filtered_mrna1, axis=0), np.max(raw_image2, axis=0), np.max(mrna2, axis=0), np.max(filtered_mrna2, axis=0), np.max(raw_image3, axis=0), np.max(mrna3, axis=0), np.max(filtered_mrna3, axis=0)]
        
        
        # Create a new figure and set the size
        fig = Figure(figsize=(10, 10))
        canvas = FigureCanvasAgg(fig)

        # Iterate through the images and add them as subplots
        for i, (image, title) in enumerate(zip(images, plot_titles)):
#             ax = fig.add_subplot(1, 1, i+1)
            ax = fig.add_subplot(3, 3, i+1)
            ax.imshow(image)
            ax.set_title(title)
            ax.axis('off')

        # Adjust the layout automatically
        fig.tight_layout()

        mRNA_path = os.path.join(mRNA_dir, os.path.basename(path) + '.png')

        # Save the plot as a PNG file with 300 DPI
        canvas.print_figure(mRNA_path, dpi=300)

    except Exception as e:
        print(f"Error processing image at path {path}: {e}")
    
    return

In [ ]:
# this code creates paths to the folders where the segmentation was bad

import os

def get_paths_from_png_files_with_bad_segmentation():
    input_dir =  fr'\\?\D:\Projects\LocProt\ISH_IF\IF_ISH_Fig3\Again'

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(input_dir):
        os.makedirs(input_dir, exist_ok=True)
        
    path_list = []

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if ".png" in file:
                # Remove the '.png' extension and the word 'Again' from the path
                modified_root = root.replace("Again", "")
                file_without_extension = os.path.splitext(file)[0]
                # Add the 'fr' prefix to the path when appending to path_list
                path_list.append(fr"{modified_root}{file_without_extension}")

    return path_list

# Get the path list with 'fr' prefix
path_list_with_bad_segmentation = get_paths_from_png_files_with_bad_segmentation()

# Print the path list to verify
for path in path_list_with_bad_segmentation:
    print(path)
print(len(path_list_with_bad_segmentation))

In [ ]:
# function of parallelization of process_image() function

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def Nikon_segment_mRNA_PARALLEL(path_list):
    with ThreadPoolExecutor(max_workers=8) as executor: # specify the number of cpu cores you want to use
        futures = {executor.submit(Nikon_segment_mRNA, path): path for path in path_list}

        for future in tqdm(as_completed(futures), total=len(path_list), desc="Processing Images"):
            pass

In [ ]:
len(path_list)

In [ ]:
Nikon_segment_mRNA_PARALLEL(path_list)

In [ ]:
Nikon_segment_mRNA_PARALLEL(path_list_with_bad_segmentation)